In [ ]:
!pip install wandb

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import wandb
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback

In [3]:
cd /content/drive/MyDrive/MachineLearning/MachineLearning10

/content/drive/MyDrive/MachineLearning/MachineLearning10


### 1- Load and preprocessing data

In [4]:
# load data

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
print(train_data.shape, test_data.shape)
train_data.info()

(1460, 81) (1459, 80)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null  

### numeric_features


In [ ]:

numeric_features = train_data.select_dtypes(include=[np.number])
numeric_features.dtypes

### categoricals features

In [ ]:
categoricals = train_data.select_dtypes(exclude=[np.number])
categoricals.describe().transpose()

### 1-2  convert categoricals features to numeric-features


In [7]:

# condition_pivot = train_data.pivot_table(index='GarageType', values='SalePrice', aggfunc=np.median)
# condition_pivot.plot(kind='bar', color='blue')
# plt.xlabel('GarageType')
# plt.ylabel('Median Sale Price')
# plt.xticks(rotation=0)
# plt.show()

train_data = train_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8]) #SaleType
test_data = test_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8]) #SaleType
train_data = train_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) #SaleCondition
test_data = test_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) #SaleCondition
train_data = train_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4]) #MSZoning
test_data = test_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4]) #MSZoning
train_data = train_data.replace(['Grvl', 'Pave'], [0, 1]) # Alley, street
test_data = test_data.replace(['Grvl', 'Pave'], [0, 1]) # Alley, street
train_data = train_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4]) # MasVnrType
test_data = test_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4]) # MasVnrType
train_data = train_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4]) # HeatingQC
test_data = test_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4]) # HeatingQC

# print (train_data.Street.value_counts())


### 1-3- calculate correlation between numerical features and labels


In [ ]:
numeric_features = train_data.select_dtypes(include=[np.number])
corr = numeric_features.corr()
corr['SalePrice'].sort_values(ascending=False)


### 1-4- selected features


In [9]:

X_train = train_data[['OverallQual', 'GrLivArea', 'GarageCars', 
                      'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath',
                      'Alley', 'TotRmsAbvGrd', 'YearBuilt',
                      'YearRemodAdd', 'GarageYrBlt', 'MasVnrArea',
                      'Fireplaces', 'BsmtFinSF1',       
                      'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
                      'OpenPorchSF', 'HalfBath', 'LotArea', 
                      'BsmtFullBath', 'BsmtUnfSF', 'SaleCondition',       
                      'BedroomAbvGr', 'GarageCond', 'ExterCond',
                      'GarageQual','ScreenPorch', ]]  
Y_train = train_data[['SalePrice']]
Y_train = np.log(Y_train)

X_test = test_data[['OverallQual', 'GrLivArea', 'GarageCars', 
                      'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath',
                      'Alley', 'TotRmsAbvGrd', 'YearBuilt',
                      'YearRemodAdd', 'GarageYrBlt', 'MasVnrArea',
                      'Fireplaces', 'BsmtFinSF1',       
                      'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
                      'OpenPorchSF', 'HalfBath', 'LotArea', 
                      'BsmtFullBath', 'BsmtUnfSF', 'SaleCondition',       
                      'BedroomAbvGr', 'GarageCond', 'ExterCond',
                      'GarageQual','ScreenPorch', ]]   

Y_test = pd.read_csv('data/sample_submission.csv')
Y_test = Y_test[['SalePrice']]
Y_test = np.log(Y_test)

print(X_test.shape, Y_test.shape)
print(X_train.shape, Y_train.shape)

(1459, 29) (1459, 1)
(1460, 29) (1460, 1)


### null columns in X_train


In [ ]:

nulls = pd.DataFrame(X_train.isnull().sum().sort_values(ascending=False)[:10])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

### null columns in X_test


In [11]:

nulls = pd.DataFrame(X_test.isnull().sum().sort_values(ascending=False)[:10])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

,Null Count
Feature,
Alley,1352
LotFrontage,227
GarageYrBlt,78
GarageCond,78
GarageQual,78
MasVnrArea,15
BsmtFullBath,2
TotalBsmtSF,1
BsmtFinSF1,1


### 1-5- fill null columns


In [ ]:

X_train['Alley'] = X_train['Alley'].fillna(2)
X_train['LotFrontage'] = X_train['Alley'].fillna(np.round(np.mean(X_train['LotFrontage']), 0))
X_train['GarageCond'] = X_train['GarageCond'].fillna(5)
X_train['GarageQual'] = X_train['GarageQual'].fillna(5)
X_train['GarageYrBlt'] = X_train['GarageYrBlt'].fillna(np.round(np.mean(X_train['GarageYrBlt']), 0))
X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(np.round(np.mean(X_train['MasVnrArea']), 0))

X_test['Alley'] = X_test['Alley'].fillna(2)
X_test['LotFrontage'] = X_test['Alley'].fillna(np.round(np.mean(X_test['LotFrontage']), 0))
X_test['GarageCond'] = X_test['GarageCond'].fillna(5)
X_test['GarageQual'] = X_test['GarageQual'].fillna(5)
X_test['GarageYrBlt'] = X_test['GarageYrBlt'].fillna(np.round(np.mean(X_test['GarageYrBlt']), 0))
X_test['MasVnrArea'] = X_test['MasVnrArea'].fillna(np.round(np.mean(X_test['MasVnrArea']), 0))
X_test['BsmtFullBath'] = X_test['BsmtFullBath'].fillna(4)
X_test['BsmtFinSF1'] = X_test['BsmtFinSF1'].fillna(np.round(np.mean(X_test['BsmtFinSF1']), 0))
X_test['TotalBsmtSF'] = X_test['TotalBsmtSF'].fillna(np.round(np.mean(X_test['TotalBsmtSF']), 0))
X_test['BsmtUnfSF'] = X_test['BsmtUnfSF'].fillna(np.round(np.mean(X_test['BsmtUnfSF']), 0))
X_test['GarageArea'] = X_test['GarageArea'].fillna(np.round(np.mean(X_test['GarageArea']), 0))
X_test['GarageCars'] = X_test['GarageCars'].fillna(5)


In [13]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1460, 29) (1460, 1) (1459, 29) (1459, 1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
wandb.init(project='MLP-House Prices', entity='nahid-ebrahimian')

### creat model and ttraining

In [76]:
model = tf.keras.models.Sequential([
                        tf.keras.layers.Dense(29, input_dim = 29, activation='relu'),
                        tf.keras.layers.Dense(64, activation='relu'),
                        tf.keras.layers.Dense(128, activation='relu'),
                        tf.keras.layers.Dense(256, activation='relu'),
                        tf.keras.layers.Dense(512, activation='relu'), 
                        tf.keras.layers.Dense(1, activation='linear')
                                    ])

In [77]:
config = wandb.config
config.learning_rate = 0.001

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
#               loss=tf.keras.losses.MAE)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MAE)


In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, 
callbacks=[WandbCallback()])

In [79]:
loss = model.evaluate(X_test, Y_test)
np.exp(loss)

46/46 [==============================] - 0s 2ms/step - loss: 0.6222


1.863028759840387